### Wesley Janson and Drew Keller
## STAT 27420 Final Project
# Modeling Code

In [1]:
# Load in relevant packages

import pandas as pd
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from data_utils import prep_features
import numpy as np

In [2]:
from load_data import data, categorical_vars, cts_vars, other_vars
# categorical_vars and cts_vars are lists of vars in each category
# other_vars are ID and date variables (categorical_vars + cts_vars + other_vars = all vars)

/Users/drewkeller/opt/anaconda3/envs/stats/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Excluding 304214     observations that did not answer 1 year price change question.


In [ ]:
# loading data from online takes ~20 seconds
# to speed up, save data locally and load from there:
DATA_PATH = '../paper_replication_data/new_data.csv'
data.to_csv(DATA_PATH)  # run this once

In [7]:
data = read_data(DATA_PATH)

NameError: name 'DATA_PATH' is not defined

In [9]:
data.treatment_bins.value_counts(dropna=False)  # check that we have a balanced dataset

0-5      163561
NaN       71082
5-10      47431
10-15     11780
20+        5376
15-20      4984
Name: treatment_bins, dtype: int64

In [4]:
data.durable_purchase.value_counts(dropna=False)  # check that we have a balanced dataset

Good          204553
Bad            71471
Neutral        12945
Don't know     12599
Refused         2646
Name: durable_purchase, dtype: int64

In [6]:
data[data.treatment_bins.isnull()].price_change_amt_next_yr.value_counts(dropna=False)

0.0    46914
NaN    24168
Name: price_change_amt_next_yr, dtype: int64

In [7]:
# prep features for modeling; use regression=True for regression models
data_regression, treatment_vars, confounder_vars = prep_features(data,regression=True)  

Excluding 15245 observations             that did not answer durable purchase question.
Excluding 21490 observations         that did not answer price change amount question.


In [10]:
data_regression.durable_purchase.value_counts(dropna=False)  # check that we have a balanced dataset

 1.0    191595
-1.0     64499
 0.0     11385
Name: durable_purchase, dtype: int64

In [11]:
len(data_regression.dropna(subset=confounder_vars))/len(data_regression)

0.7146280642592503

In [12]:
data_regression = data_regression.dropna(subset=confounder_vars)

In [13]:
# First model, ordered probit-same as Bachmann et al.
mod_prob = OrderedModel(data_regression['durable_purchase'],
                        data_regression[confounder_vars+treatment_vars],
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

In [ ]:
len(data_regression.dropna(subset=["price_change_amt_next_yr"]))/len(data_regression)

0.8450075753873222

In [ ]:
# First model, ordered probit-same as Bachmann et al.
mod_prob = OrderedModel(data_regression['durable_purchase'],
                        data_regression[confounder_vars+["price_change_amt_next_yr"]],
                        distr='probit')

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

MissingDataError: exog contains inf or nans

In [ ]:
# Second model - some sort of XGBoost?